# Описание проекта "Исследование надёжности заёмщиков"

Заказчик — кредитный отдел банка. На основе статистики о платёжеспособности клиентов исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.


## Откройте файл с данными и изучите общую информацию

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
from pymystem3 import Mystem

In [2]:
# Прочитаем файл с данными и сохраним в  переменную data.
data = pd.read_csv('data.csv')

In [5]:
# Выведим первых 5 строк data.
data.head()

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [6]:
# Посмотрим общую информацию о данных в data.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        21525 non-null  int64  
 1   children          21525 non-null  int64  
 2   days_employed     19351 non-null  float64
 3   dob_years         21525 non-null  int64  
 4   education         21525 non-null  object 
 5   education_id      21525 non-null  int64  
 6   family_status     21525 non-null  object 
 7   family_status_id  21525 non-null  int64  
 8   gender            21525 non-null  object 
 9   income_type       21525 non-null  object 
 10  debt              21525 non-null  int64  
 11  total_income      19351 non-null  float64
 12  purpose           21525 non-null  object 
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


### Описание данных
`children` — количество детей в семье   
`days_employed` — общий трудовой стаж в днях  
`dob_years` — уровень образования клиента  
`education` — город пользователя    
`education_id` — идентификатор уровня образования  
`family_status` — семейное положение  
`family_status_id` — идентификатор семейного положения  
`gender` — пол клиента  
`income_type` — тип занятости  
`debt`  — имел ли задолженность по возврату кредитов  
`total_income` — ежемесячный доход  
`purpose` — цель получения кредита  

**Вывод**

Импортировали библиотеку Pandas. Прочитали файл с данными и сохранили его в переменной `data`. Вывели первые 10 значений для знакомства с таблицей. Наблюдаем несколько типов данных (int64 / float64 / object) и пропуски в `days_employed` и `total_income`. 

## Предобработка данных

### Обработка пропусков

In [7]:
# Посмотрим на тип пропусков
nan_values = data[data['days_employed'].isna()]
nan_values.head()

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Пропуски имелись в столбцах `days_employed` и `total_income` по типу NaN  
Причина пропуска может быть связанна с тем, что клиент не предоставил сведения о работе и о ЗП.

In [8]:
# Заменим пропуски на медиану, так как без предоставленных сведений о трудовом стаже мы не можем корректно судить о надежности заемщика.
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
# Заменим пропуски на медиану, так как без предоставленных сведений о ЗП мы не можем корректно судить о надежности заемщика.
data['total_income'] = data['total_income'].fillna(data['total_income'].median()) 

In [9]:
# Проверим остались ли пропуски.
print(data.isna().sum())

Unnamed: 0          0
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Проверим данные на уникальность

In [10]:
#Проверям коллонку с детьми на уникальные значения, чтобы найти возможные отклонения.
children_unique = data['children'].unique()
print(children_unique)

[ 1  0  3  2 -1  4 20  5]


Мы видим два отклонения (Отрицательное количество детей и 20 детей). Проработаем данные пункты.

In [11]:
# Уберем отрицательное количесво детей. (с логикой, что это опечатка)
data['children'] = abs(data['children'])
# Заменим на возможное значение. (с логикой, что это опечатка)
data.loc[data['children'] == 20, 'children'] = 2
# Снова проверим на уникальность.
children_unique = data['children'].unique()
print(children_unique)

[1 0 3 2 4 5]


In [12]:
# Смотрим минимальное и максимальное значение столбца `days_employed`.
print(data['days_employed'].describe().astype('int'))

count     21525
mean      56557
std      134922
min      -18388
25%       -2518
50%       -1203
75%        -385
max      401755
Name: days_employed, dtype: int64


Мы видим слишком большую амплитуду, а так же отрицательное значение в отработанных дня. Надо предположить, что это могут быть за значения.

In [13]:
# Посмотрим самые большие значения (предположительно, они связанны с пенсионерами)
data.sort_values(by='days_employed', ascending=False).head(10)

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
4697,4697,0,401635.032697,56,среднее,1,женат / замужем,0,F,пенсионер,0,48242.322502,покупка недвижимости
13420,13420,0,401619.633298,63,Среднее,1,гражданский брак,1,F,пенсионер,0,51449.788325,сыграть свадьбу
17823,17823,0,401614.475622,59,среднее,1,женат / замужем,0,F,пенсионер,0,152769.694536,покупка жилья для сдачи
10991,10991,0,401591.828457,56,среднее,1,в разводе,3,F,пенсионер,0,39513.517543,получение дополнительного образования
8369,8369,0,401590.452231,58,среднее,1,женат / замужем,0,F,пенсионер,0,175306.312902,образование


In [14]:
# Изменим все отрицательные значения на положительные.
data['days_employed'] = abs(data['days_employed'])

**Вывод**

Мы обнаружили пропущенные данные в двух столбцах `days_employed` и `total_income`. Пропуски заменили на 0, и следом удалили строки с пропусками, так как они не корректны.

### Замена типа данных

In [15]:
# Заменим столбцы с типом данных float64 на тип данных int методом astype()
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

**Вывод**

Все столбцы с типом данных `float64` заменили на тип данных `int64` методом astype()

### Обработка дубликатов

In [16]:
# Приведем все данные в столбце education к нижнему регитру.
data['education'] = data['education'].str.lower()
# Приведем все данные в столбце family_status к нижнему регитру.
data['family_status'] = data['family_status'].str.lower()
# Удалим дубликаты, если такие есть.
data = data.drop_duplicates() 
data.duplicated().sum()

0

**Вывод**

Мы привели все значения к одному регистру и после удалили все дубликаты.

### Лемматизация

In [17]:
from pymystem3 import Mystem
m = Mystem()

# Создадим функцию, которая проведет лемматизацию
def purpose_change(purpose):
    lemma = m.lemmatize(purpose)
    if 'автомобиль' in lemma:
        return 'автомобиль'           
    elif 'образование' in lemma:
        return 'образование'
    elif 'свадьба' in lemma:
        return 'свадьба'
    elif 'ремонт' or 'строительство' or 'недвижимость' in lemma:
        return 'жилье'    
    else:
        return 'другое'
# Применим данную функцию к столбцу purpose ДФ и создадим новый столбец purpose_name.
data['purpose_name'] = data['purpose'].apply(purpose_change) 
data.head()   

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_name
0,0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**

Мы провели лемматизацию столбца 'purpose' и выделили 4 основных категорий (автомобиль, жилье, образование, свадьба)

### Категоризация данных

In [18]:
# Создаем "словарь" с количеством детей, семейным положением, уровнем дохода и целями кредита.
data_log = data[['children', 'family_status', 'debt', 'total_income', 'purpose_name']]
data_log.head()

,children,family_status,debt,total_income,purpose_name
0,1,женат / замужем,0,253875,жилье
1,1,женат / замужем,0,112080,автомобиль
2,0,женат / замужем,0,145885,жилье
3,3,женат / замужем,0,267628,образование
4,0,гражданский брак,0,158616,свадьба


In [19]:
# Создадим функцию, которая катигоризирует нам столбец children
data_log = data_log.copy()
def children(qty):
    if qty == 0:
        return 'нет детей'
    if qty >= 1 and qty <= 2:
        return '1-2 ребенка'
    return 'многодетная семья'

# Применим данную функцию к столбцу children ДФ и создадим новый столбец children_status.
data_log['children_status'] = data_log['children'].apply(children)
data_log.head()

,children,family_status,debt,total_income,purpose_name,children_status
0,1,женат / замужем,0,253875,жилье,1-2 ребенка
1,1,женат / замужем,0,112080,автомобиль,1-2 ребенка
2,0,женат / замужем,0,145885,жилье,нет детей
3,3,женат / замужем,0,267628,образование,многодетная семья
4,0,гражданский брак,0,158616,свадьба,нет детей


In [20]:
# Для категоризации по уровню дохода, определим суммы зарплат на 25%/50%/75% 
data_log['total_income'].describe().astype('int')

count      21525
mean      165158
std        97866
min        20667
25%       107798
50%       145017
75%       195543
max      2265604
Name: total_income, dtype: int64

In [21]:
# Выделим 4 основные группы: Минимальный(<25%), Ниже среднего (25%-50%), Средний (50%-75%), Высокий(>75%)
data_log = data_log.copy()
def income(total):
    if total <= 108000:
        return 'минимальный'
    if total >= 108001 and total <= 145000:
        return 'ниже среднего'
    if total >= 145001 and total <= 195000:
        return 'средний'
    return 'высокий'

# Применим данную функцию к столбцу total_income ДФ и создадим новый столбец income_status.
data_log['income_status'] = data_log['total_income'].apply(income)
data_log.head()

,children,family_status,debt,total_income,purpose_name,children_status,income_status
0,1,женат / замужем,0,253875,жилье,1-2 ребенка,высокий
1,1,женат / замужем,0,112080,автомобиль,1-2 ребенка,ниже среднего
2,0,женат / замужем,0,145885,жилье,нет детей,средний
3,3,женат / замужем,0,267628,образование,многодетная семья,высокий
4,0,гражданский брак,0,158616,свадьба,нет детей,средний


In [22]:
# Создадим "словарь" с количеством детей, семейным положением, уровнем дохода и целями кредита.
data_new = data_log[['family_status', 'debt', 'purpose_name', 'children_status', 'income_status']]
data_new.head()

,family_status,debt,purpose_name,children_status,income_status
0,женат / замужем,0,жилье,1-2 ребенка,высокий
1,женат / замужем,0,автомобиль,1-2 ребенка,ниже среднего
2,женат / замужем,0,жилье,нет детей,средний
3,женат / замужем,0,образование,многодетная семья,высокий
4,гражданский брак,0,свадьба,нет детей,средний


**Вывод**

Мы провели категоризацию данных таких столбцов, как `children`, `total_income` выделив их условные статусы.

## Ответьтим на возможные вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [18]:
data_new.groupby('children_status')['debt'].mean().sort_values()

children_status
нет детей            0.075438
многодетная семья    0.081579
1-2 ребенка          0.092654
Name: debt, dtype: float64

**Вывод**

Исходя из данного иследования мы видим, что заемщики без детей являются самыми надежными, их доля составляет всего 0.075, а самым не надежным заемщиком является категория людей у которых 1-2 ребенка и их доля составляет  0.092

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [19]:
data_new.groupby('family_status')['debt'].mean().sort_values()

family_status
вдовец / вдова           0.065693
в разводе                0.071130
женат / замужем          0.075452
гражданский брак         0.093471
не женат / не замужем    0.097509
Name: debt, dtype: float64

**Вывод**

Мы можем наблюдать, что чаще всего возвращают кредиты во время такая категории людей с семейным статусом 'вдовец / вдова'(доля 0.066), хуже всего это делают люди в статусе 'не женат / не замужем'(доля 0.097).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [20]:
data_new.groupby('income_status')['debt'].mean().sort_values()

income_status
высокий          0.071732
минимальный      0.079653
средний          0.086204
ниже среднего    0.087497
Name: debt, dtype: float64

**Вывод**

Мы можем наблюдать, что чаще всего возвращают кредиты своевременно заемщики, которые имеют 'высокий' доход (доля 0.072), хуже всего это делают заемщики с доходом 'ниже среднего' (доля 0.087).

- Как разные цели кредита влияют на его возврат в срок?

In [21]:
data_new.groupby('purpose_name')['debt'].mean().sort_values()

purpose_name
жилье          0.072334
свадьба        0.080034
образование    0.092200
автомобиль     0.093590
Name: debt, dtype: float64

**Вывод**

Группируя результаты по целям кредита, мы можем наблюдать, что чаще всего возвращают кредиты которые берут на 'жилье'(доля 0.072), хуже всего возвращаются кредиты на 'автомобиль' (доля 0.093).

## Общий вывод

Анализируя датасет:
- С помощью метода .isna() были найдены пропущенные значения в столбцах 'days_employed' и 'total_income' по типу NaN, которые были устранены методом .fillna() на медианные значения.
- В столбце data['children'] с помощью метода .unique() были найдены такие артифакты, как отрицательное количесво детей (-1), которые были исправлены мотодом abs(), и так же заемщики с 20-ю детьми. Данные значения сокращенны до 2 через метод точечного изменения данных ячейки (data.loc[data['children'] == 20, 'children'] = 2
- В столбце ['days_employed'] с помощью метода .describe(), где обнаруживаем такие артифакты, как отрицательное количесво дней стажа, и которое так же исправляем методом abs(). Еще находим большие значения в количесве дней стажа, но их не отработали, так как эт данные нам не важны для исследования (к ним можно было применить уменьшающий коэффициент исходя из минимального отрицаельного значения, по моим подсчетам данный коэффициент должен был быть /22).
- В столбцах data['days_employed'] и data['total_income'] мы меняем тип данных с помощью метода .astype('int').
- В столбце ['education'] и data['family_status'] мы наблюдаем разный регистр который устроняем путем метода .str.lower(), что приводит все к нижнему регистру. Далее избавляемся от возможных дубликатов методом .drop_duplicates().
- Проводим процесс Лемматизации данных. Для этого импортируем Mystem из библиотеки pymystem3 и создаем функцию, которая нам сортирует данные на 4 варианта. И мы добавляем новый столбец с этими данными.
- Перехотя к Категоризации данных мы создаем словарь data_log в котором оставляем только нужные для исследования столбцы. Далее с помощью написанных функций, мы категоризируем данные и применяем данную функцию к новому столбцу методом .apply(). Создаем финальный словарь data_new, где собраны все необходимы для ислледования столбцы.

Переходя к итогам исследования:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Да, мы можем наблюдать зависимость: Заемщики без детей являются самыми надежными, их доля составляет всего 0.075, а самым не надежным заемщиком является категория людей у которых 1-2 ребенка и их доля составляет  0.092
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Да, мы можем наблюдать зависимость: Чаще всего возвращают кредиты во время такая категории людей с семейным статусом 'вдовец / вдова'(доля 0.066), хуже всего это делают люди в статусе 'не женат / не замужем'(доля 0.097)
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Да, мы можем наблюдать зависимость: чаще всего возвращают кредиты своевременно заемщики, которые имеют 'высокий' доход (доля 0.072), хуже всего это делают заемщики с доходом 'ниже среднего' (доля 0.087)
4. Как разные цели кредита влияют на его возврат в срок?
- Да, мы можем наблюдать зависимость: Чаще всего возвращают кредиты которые берут на 'жилье'(доля 0.072), хуже всего возвращаются кредиты на 'автомобиль' (доля 0.093)

Мы можем сделать общий вывод такой, что самый благонадежный заемщик это тот, у которого есть дети, он находится в разводе или вдовец, с высоким уровнем дохода и цель его кредита на ремонт.

In [22]:
data_new.groupby(['children_status','family_status','income_status','purpose_name'])['debt'].mean().sort_values(ascending=False).reset_index()

,children_status,family_status,income_status,purpose_name,debt
0,многодетная семья,не женат / не замужем,минимальный,жилье,1.000000
1,многодетная семья,гражданский брак,высокий,автомобиль,1.000000
2,многодетная семья,гражданский брак,минимальный,свадьба,0.666667
3,многодетная семья,не женат / не замужем,средний,жилье,0.500000
4,1-2 ребенка,вдовец / вдова,ниже среднего,образование,0.400000
...,...,...,...,...,...
169,многодетная семья,гражданский брак,ниже среднего,свадьба,0.000000
170,многодетная семья,гражданский брак,средний,жилье,0.000000
171,многодетная семья,женат / замужем,высокий,образование,0.000000
172,многодетная семья,женат / замужем,минимальный,образование,0.000000
